# Writing Pandas DataFrames to Excel workbooks

I used to never really consider Excel as a file format for writing the output of analysis. I think this was mainly because it's hard to preview the files without proprietary software. However, for explicit uses, Excel makes a lot of sense. It has some support for typing and formats and, most importantly, it lets you have multiple tables of data in a single file. When sharing data with reporters who primarily work with data in Excel, I find it really helpful to be able to send a README and a data dictionary along with the data.

I had writen data using [`to_excel()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html), but this notebook includes some things I learned to support writing Excel files from Pandas in a way that made them seem like ones I meticulously formatted by hand. 

In [1]:
# Setup

import copy

from openpyxl.utils import get_column_letter
import pandas as pd

import settings

## To write multiple worksheets, you need to used a `pd.ExcelWriter` object

In [2]:
# Make some toy data

df = pd.DataFrame(
    [
        ['a', 'b'],
        ['c', 'd'],
    ],
    index=['row 1', 'row 2'],
    columns=['col 1', 'col 2'],
)

df

,col 1,col 2
row 1,a,b
row 2,c,d


One thing I like to do when writing an Excel workbook from a Pandas `DataFrame` is to also add a `README` worksheet to describe the data.

In [3]:
# Create a DataFrame suitable for writing a README worksheet

readme = pd.DataFrame([
    {
        "heading1": "What is this?",
        "heading2": "",
        "content": "This is a sample dataset to illustrate writing to Excel files from Pandas",
    }
])

readme

,heading1,heading2,content
0,What is this?,,This is a sample dataset to illustrate writing...


In order to write multiple worksheets to one Excel workbook, you have to used an [`ExcelWriter`](https://pandas.pydata.org/docs/reference/api/pandas.ExcelWriter.html#pandas.ExcelWriter) object.

It works as a context manager.

In [4]:
# Write data and a README to one Excel workbook

os.makedirs(settings.DATA_DIR_PROCESSED, exist_ok=True)

excel_out_path = settings.DATA_DIR_PROCESSED / "sample_excel_data.xlsx"

with pd.ExcelWriter(excel_out_path) as writer:
    df.to_excel(writer, index=False, sheet_name="data")
    readme.to_excel(writer, index=False, sheet_name="README")

## Sizing columns

Many times I want to resize the columns to fit the contents. The key to doing is is accessing the underlying [`Worksheet`](https://openpyxl.readthedocs.io/en/stable/api/openpyxl.worksheet.worksheet.html#openpyxl.worksheet.worksheet.Worksheet) object and configuring various properties of the columns and cells. The worksheets can be accessed from the `.sheets` attribute of the `ExcelWriter` object.

Rather than getting into the full API, here are a couple of functions I wrote in a project to help format the worksheets.

In [5]:
# Define some functions

def size_excel_columns(df, worksheet, wrap_columns=None):
    """Size the columns to fit the values in an Excel worksheet corresponding to a DataFrame"""
    # h/t https://stackoverflow.com/a/40535454/386210
    if wrap_columns is None:
        wrap_columns = []

    for idx, col in enumerate(df):  # loop through all columns
        col_data = df[col]
        if col in wrap_columns:
            mean_len = (
                max(
                    col_data.astype(str).map(len).mean(),
                    len(str(col_data.name)),  # len of column name/header
                )
                + 1  # adding a little extra space
            )
            col_letter = get_column_letter(idx + 1)
            worksheet.column_dimensions[col_letter].width = mean_len
            for row_idx in range(len(col_data)):
                cell_label = f"{col_letter}{row_idx + 2}"
                alignment = copy.copy(worksheet[cell_label].alignment)
                alignment.wrapText = True
                worksheet[cell_label].alignment = alignment

        else:
            max_len = (
                max(
                    (
                        col_data.astype(str).map(len).max(),  # len of largest item
                        len(str(col_data.name)),  # len of column name/header
                    )
                )
                + 1  # adding a little extra space
            )
            worksheet.column_dimensions[get_column_letter(idx + 1)].width = max_len


def set_excel_column_width(df, worksheet, column_name, width, wrap=False):
    """
    Explicitly set the column width of an Excel sheet

    """
    col_letter = get_column_letter(df.columns.get_loc(column_name) + 1)
    worksheet.column_dimensions[col_letter].width = width
    for row_idx in range(df.shape[0]):
        cell_label = f"{col_letter}{row_idx + 2}"
        alignment = copy.copy(worksheet[cell_label].alignment)
        alignment.wrapText = wrap
        worksheet[cell_label].alignment = alignment

In [6]:
# Use worksheet column width/wrapping setting functions

with pd.ExcelWriter(excel_out_path) as writer:
    df.to_excel(writer, index=False, sheet_name="data")
    size_excel_columns(df, writer.sheets["data"])
    readme.to_excel(writer, index=False, sheet_name="README")
    set_excel_column_width(readme, writer.sheets["README"], "content", 200, wrap=True)

## Resources

- [Working with Pandas and NumPy](https://openpyxl.readthedocs.io/en/stable/pandas.html) (OpenPyXL)
